In [1]:
# 📁 Part 1: Setting up our smart question-answering system
# Let's build an AI that can read text and answer questions about it!

import torch
import logging
from datetime import datetime
from typing import List, Tuple

# Set up logging so we can see what's happening
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger("SmartQA")

print("🧠 Welcome to our Smart Question Answering System!")
print("Let me get everything set up for you...")

# First, let's make sure we have all the tools we need
try:
    from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
    from datasets import load_dataset
    print("✅ Great! All tools are loaded and ready.")
except ImportError as e:
    print(f"❌ Oops! Missing some tools: {e}")
    print("Please run: pip install transformers datasets torch")
    exit()

class SimpleQAConfig:
    """Just some settings to make our system work well"""
    def __init__(self):
        self.model_name = "distilbert-base-uncased-distilled-squad"  # A smart, fast model
        self.max_length = 512  # How much text we can read at once
        self.batch_size = 8    # How many questions we can process together

class Answer:
    """A simple way to store our answers"""
    def __init__(self, text, confidence, context, question, time_taken):
        self.text = text
        self.confidence = confidence
        self.context = context[:100] + "..." if len(context) > 100 else context
        self.question = question
        self.time_taken = time_taken
        self.timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    def display(self):
        """Show the answer in a nice, readable way"""
        print(f"\n🎯 ANSWER: {self.text}")
        print(f"📊 Confidence: {self.confidence:.1%}")
        print(f"⏱️  Found in: {self.time_taken:.2f} seconds")
        if self.confidence < 0.3:
            print("💡 Hint: This answer might not be very reliable")
        elif self.confidence < 0.7:
            print("💡 Hint: This answer seems pretty good")
        else:
            print("💡 Hint: This answer looks very confident!")

🧠 Welcome to our Smart Question Answering System!
Let me get everything set up for you...
✅ Great! All tools are loaded and ready.


In [2]:
# 📁 Part 2: The Brain - Loading our AI model
# This is where the magic happens!

class QABrain:
    """The smart part that actually understands and answers questions"""

    def __init__(self, config):
        self.config = config
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"🔧 Using: {self.device.upper()} for processing")

        self.setup_model()

    def setup_model(self):
        """Get our AI model ready to work"""
        print("🔄 Loading the AI model...")
        try:
            # This is like loading a trained brain that knows how to answer questions
            self.qa_pipeline = pipeline(
                "question-answering",
                model=self.config.model_name,
                tokenizer=self.config.model_name,
                device=0 if self.device == "cuda" else -1
            )
            print("✅ AI model loaded successfully!")
            print("🤖 The system is now ready to answer your questions!")
        except Exception as e:
            print(f"❌ Failed to load the model: {e}")
            print("💡 Please check your internet connection and try again")
            raise

    def find_answer(self, question: str, context: str) -> Answer:
        """
        The main magic happens here!
        Given a question and some text, find the answer.
        """
        start_time = datetime.now()

        if not question.strip():
            return Answer("Please ask a question!", 0.0, context, question, 0.0)

        if not context.strip():
            return Answer("Please provide some text to search in!", 0.0, context, question, 0.0)

        print(f"🤔 Thinking about: '{question}'")

        try:
            # Ask the AI to find the answer in the text
            result = self.qa_pipeline(
                question=question,
                context=context,
                max_answer_len=100,  # Don't make answers too long
                handle_impossible_answer=True  # Be honest if we can't find it
            )

            time_taken = (datetime.now() - start_time).total_seconds()

            # If the AI isn't confident, let us know
            if result['score'] < 0.1:
                result['answer'] = "I'm not very confident, but I think: " + result['answer']

            return Answer(
                text=result['answer'],
                confidence=result['score'],
                context=context,
                question=question,
                time_taken=time_taken
            )

        except Exception as e:
            print(f"❌ Oops! Something went wrong: {e}")
            time_taken = (datetime.now() - start_time).total_seconds()
            return Answer(
                text="Sorry, I encountered an error while processing your question.",
                confidence=0.0,
                context=context,
                question=question,
                time_taken=time_taken
            )

In [3]:
# 📁 Part 3: Making it easy to use
# Let's create a simple way for humans to interact with our system

class SimpleQASystem:
    """
    A friendly question-answering system that anyone can use!
    Just give it some text and ask questions.
    """

    def __init__(self):
        print("\n" + "="*50)
        print("   INITIALIZING SMART QA SYSTEM")
        print("="*50)

        self.config = SimpleQAConfig()
        self.brain = QABrain(self.config)

        print("\n🌟 System Ready! Here's what I can do:")
        print("   • Read any text document you provide")
        print("   • Answer questions about that text")
        print("   • Tell you how confident I am in each answer")
        print("   • Work with all kinds of topics!")

    def ask_question(self, question: str, context: str) -> Answer:
        """The main way to ask questions"""
        return self.brain.find_answer(question, context)

    def chat_mode(self):
        """A friendly chat interface for asking multiple questions"""
        print("\n" + "💬" * 20)
        print("   CHAT MODE ACTIVATED")
        print("💬" * 20)
        print("\nHi! I'm your reading assistant. I can help you find information in texts.")
        print("First, let me know what you'd like me to read.")

        # Get the text to analyze
        print("\n📖 Please paste your text below (press Enter twice when done):")
        lines = []
        while True:
            line = input()
            if line == "" and len(lines) > 0:
                break
            lines.append(line)

        context = "\n".join(lines)

        if not context.strip():
            print("❌ You didn't provide any text. Let's try again later!")
            return

        print(f"✅ Great! I've read your text ({len(context)} characters)")
        print("\nNow you can ask me questions about this text!")
        print("Type 'quit' to exit, or 'new text' to start over")

        # Keep answering questions until the user wants to stop
        while True:
            print("\n" + "-" * 40)
            question = input("\n❓ Your question: ").strip()

            if question.lower() == 'quit':
                print("👋 Thanks for chatting! Goodbye!")
                break
            elif question.lower() == 'new text':
                print("🔄 Let's start with a new text!")
                self.chat_mode()
                break
            elif not question:
                print("💡 Please type a question or 'quit' to exit")
                continue

            # Get and display the answer
            answer = self.ask_question(question, context)
            answer.display()

In [4]:
# 📁 Part 4: Let's test our system with some examples
# This shows how the system works in real life

def run_demo_examples():
    """Show what our system can do with some prepared examples"""

    print("\n" + "🎯" * 20)
    print("   DEMONSTRATION TIME!")
    print("🎯" * 20)

    # Initialize our system
    qa_system = SimpleQASystem()

    # Example 1: Technology
    tech_context = """
    Artificial Intelligence (AI) is transforming how we live and work. AI systems
    can now recognize speech, identify images, and even drive cars. Machine learning,
    which is a part of AI, allows computers to learn from data without being
    explicitly programmed. Major tech companies like Google, Amazon, and Microsoft
    are investing heavily in AI research.
    """

    tech_questions = [
        "What can AI systems do?",
        "What is machine learning?",
        "Which companies are investing in AI?"
    ]

    print(f"\n1. 🤖 TECHNOLOGY EXAMPLE")
    print(f"   Context: {tech_context[:80]}...")

    for i, question in enumerate(tech_questions, 1):
        print(f"\n   Question {i}: {question}")
        answer = qa_system.ask_question(question, tech_context)
        answer.display()

    # Example 2: History
    history_context = """
    The Renaissance was a period in European history between the 14th and 17th centuries.
    It marked the transition from the Middle Ages to modernity. The Renaissance began
    in Florence, Italy and was characterized by a revival of classical learning and wisdom.
    Famous artists from this period include Leonardo da Vinci and Michelangelo.
    """

    history_questions = [
        "When was the Renaissance?",
        "Where did the Renaissance begin?",
        "Who were famous Renaissance artists?"
    ]

    print(f"\n2. 📚 HISTORY EXAMPLE")
    print(f"   Context: {history_context[:80]}...")

    for i, question in enumerate(history_questions, 1):
        print(f"\n   Question {i}: {question}")
        answer = qa_system.ask_question(question, history_context)
        answer.display()

def test_with_squad_dataset():
    """Test our system with the standard SQuAD dataset"""
    print("\n" + "🧪" * 20)
    print("   TESTING WITH REAL DATA")
    print("🧪" * 20)

    print("Loading the SQuAD dataset (this might take a moment)...")
    try:
        # Load a small sample of the famous SQuAD dataset
        dataset = load_dataset("squad", split="validation[:10]")  # Just 10 examples
        qa_system = SimpleQASystem()

        print(f"✅ Loaded {len(dataset)} test questions")
        print("\nLet's see how our system handles real test questions!")

        correct_answers = 0
        total_questions = len(dataset)

        for i, example in enumerate(dataset, 1):
            print(f"\n--- Question {i}/{total_questions} ---")
            print(f"Question: {example['question']}")

            answer = qa_system.ask_question(
                example['question'],
                example['context']
            )

            # Check if the answer is correct (simple version)
            actual_answer = example['answers']['text'][0]
            if answer.text.lower() in actual_answer.lower() or actual_answer.lower() in answer.text.lower():
                correct_answers += 1
                print("✅ Correct!")
            else:
                print(f"❌ Close! Actual answer: {actual_answer}")

            answer.display()

        accuracy = correct_answers / total_questions
        print(f"\n📊 FINAL SCORE: {correct_answers}/{total_questions} ({accuracy:.1%})")

    except Exception as e:
        print(f"❌ Couldn't load the test dataset: {e}")
        print("💡 This is okay - we can still use the system with our own texts!")

In [5]:
# 📁 Part 5: The main program - bringing it all together!

def main():
    """The main function that runs our entire system"""

    print("🌟 SMART QUESTION ANSWERING SYSTEM 🌟")
    print("======================================")

    while True:
        print("\nWhat would you like to do?")
        print("1. 🚀 Try chat mode (recommended for beginners)")
        print("2. 🎯 See demo examples")
        print("3. 🧪 Run accuracy tests")
        print("4. ❌ Exit")

        choice = input("\nEnter your choice (1-4): ").strip()

        if choice == "1":
            # Start the interactive chat mode
            qa_system = SimpleQASystem()
            qa_system.chat_mode()

        elif choice == "2":
            # Show the demo examples
            run_demo_examples()

        elif choice == "3":
            # Run the accuracy tests
            test_with_squad_dataset()

        elif choice == "4":
            print("👋 Thanks for using our Smart QA System! Goodbye!")
            break

        else:
            print("❌ Please enter 1, 2, 3, or 4")

# Some helper functions for common tasks
def quick_answer(question: str, context: str) -> None:
    """
    Quick function for when you just want one answer fast
    """
    qa_system = SimpleQASystem()
    answer = qa_system.ask_question(question, context)
    answer.display()

if __name__ == "__main__":
    # This runs when you execute the script directly
    main()

🌟 SMART QUESTION ANSWERING SYSTEM 🌟

What would you like to do?
1. 🚀 Try chat mode (recommended for beginners)
2. 🎯 See demo examples
3. 🧪 Run accuracy tests
4. ❌ Exit

Enter your choice (1-4): 3

🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪
   TESTING WITH REAL DATA
🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪🧪
Loading the SQuAD dataset (this might take a moment)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

plain_text/validation-00000-of-00001.par(…):   0%|          | 0.00/1.82M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cuda:0


✅ AI model loaded successfully!
🤖 The system is now ready to answer your questions!

🌟 System Ready! Here's what I can do:
   • Read any text document you provide
   • Answer questions about that text
   • Tell you how confident I am in each answer
   • Work with all kinds of topics!
✅ Loaded 10 test questions

Let's see how our system handles real test questions!

--- Question 1/10 ---
Question: Which NFL team represented the AFC at Super Bowl 50?
🤔 Thinking about: 'Which NFL team represented the AFC at Super Bowl 50?'
✅ Correct!

🎯 ANSWER: Denver Broncos
📊 Confidence: 98.8%
⏱️  Found in: 0.39 seconds
💡 Hint: This answer looks very confident!

--- Question 2/10 ---
Question: Which NFL team represented the NFC at Super Bowl 50?
🤔 Thinking about: 'Which NFL team represented the NFC at Super Bowl 50?'
✅ Correct!

🎯 ANSWER: Carolina Panthers
📊 Confidence: 99.8%
⏱️  Found in: 0.01 seconds
💡 Hint: This answer looks very confident!

--- Question 3/10 ---
Question: Where did Super Bowl 50 tak